In [35]:
import glob
import pandas as pd
import os
from thefuzz import fuzz
from thefuzz import process
import json

In [2]:
files = glob.glob('source_configurations/new_sources/*.*')

In [4]:
df = pd.DataFrame(list(files), columns=['file_path'])
df['file_name'] = df['file_path'].apply(lambda x: os.path.basename(x))
df['temp'] = df.file_name.apply(lambda value: os.path.splitext(value))
df['fname'] = df.apply(lambda row: row['temp'][0], axis=1)
df['extension'] = df.apply(lambda row: row['temp'][1], axis=1)
df.drop('temp', axis=1, inplace=True, errors='ignore')
df.drop('file_name', axis=1, inplace=True, errors='ignore')

In [8]:
collections = pd.read_json('../sde_collections/fixtures/collections.json')
# normalize the JSON data and convert it into a DataFrame
json_df = pd.json_normalize(collections['fields'])

# join the two DataFrames
collections = collections.join(json_df)
collections.drop('fields', axis=1, errors='ignore', inplace=True)

In [16]:
file_names = df['fname'].str.strip().to_list()
db_collection_names = collections.name.str.strip().to_list()
# 17 out of 37 did not match, 20 matched
unmatched = set(file_names) - set(db_collection_names)
unmatched

match_dict = {item: process.extract(item, db_collection_names, limit=1)[0][0] for item in unmatched}

In [17]:
def match_name(name):
    if name in match_dict.keys():
        return match_dict[name]
    else:
        return name

df['collection_name'] = df['fname'].apply(lambda x: match_name(x))

In [20]:
df = df.merge(collections, left_on='collection_name', right_on='name', how='left')

In [23]:
def load_file(file_path):
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith('.xlsx'):
        return pd.read_excel(file_path)

def populated(row):
    subdf = load_file(row)
    try:
        return subdf['Index?'].any()
    except KeyError:
        return False

df['populated'] = df['file_path'].apply(lambda x: populated(x))

/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (3,4,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [26]:
df = df[df['populated'] == True]

In [37]:
def generate_json(row, config_folder):
    return {
        "model": "sde_collections.candidateurl",
        "fields": {
            "collection": collection_ids[config_folder],
            "url": row.url,
            "scraped_title": row.title,
            "generated_title": "",
            "level": 2,
            "visited": False
        }
    }

replacement_dict = {
    'pds4_documents': 'PDS_PDS4_Documents_Website',
    'heliophysics_events_knowledgebase': 'Helio_Events_Knowledgebase_Website',
    'mars_target_encyclopedia': 'mars_target_encyclopedia_mte',
    'pds_near_earth_asteroid_rendezvous_near_data_archive_website': 'PDS_Near_Earth_Asteroid_Rendezvous_(NEAR)_Data_Archive_Website',
    'solar_system_exploration_research_virtual_institute_sservi': 'PDS_Solar_System_Exploration_Research_Virtual_Institute_(SSERVI)_Website',
    'astromaterials_acquisition_and_curation_office': 'PDS_Astromaterials_Acquisition_and_Curation_Office_Website',
    'mars_exploration_program': 'PDS_Mars_Exploration_Program_Website',
    'nasa_science_earths_moon': 'PDS_NASA_Science_Earths_Moon_Website',
}

for item in df.iterrows():
    config_folder = item[1]['config_folder']
    subdf = load_file(item[1]['file_path'])
    subdf.rename(columns={'Full Url': 'url', 'Scraped Title': 'title'}, inplace=True)
    subdf = subdf[['url', 'title']]
    subdf = subdf.drop_duplicates(subset=['url'])

    directory = f'../scraper/scraped_urls/{config_folder}'
    json_file_name = f"{directory}/urls.json"
    if not os.path.exists(directory):
        os.makedirs(directory)
    if not config_folder in collection_ids:
        try:
            config_folder = replacement_dict[config_folder]
        except KeyError:
            print(f"Skipping {config_folder}")
            continue
    subdf['json'] = subdf.apply(generate_json, args=(config_folder, ), axis=1)

    with open(json_file_name, "w") as f:
        f.write(json.dumps(subdf['json'].to_list(), indent=4))

PDS_Astropedia_Lunar_and_Planetary_Cartographic_Catalog_Website
gcn_circulars
igwn_public_alerts_user_guide
general_coordinates_network_gcn
nasa_sea_level_change
giss_publication_list
PDS_Recently_Archived_Volumes_Website


/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


PDS_Cassini_Resource_Page_Website


/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


emac_exoplanet_modeling_and_analysis_center
earth_observer_publications
algorithm_theoretical_basis_documents
goddard_institute_for_space_studies
giss_datasets_and_derived_materials
GENELAB_Publications_Website
exoplanet_opacities_database
our_changing_planet_the_view_from_space_images
nasa_wavelength
PDS_NASA_Science_Solar_System_Exploration_Website


/tmp/ipykernel_32301/366141159.py:3: DtypeWarning: Columns (3,4,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


giss_software_tools
PDS_Small_Bodies_Data_Ferret_Website
nasa_global_climate_change
gcn_missions_instruments_and_facilities
nasa_power


In [31]:
collection_ids = {'nasa_power': 931, 'emac_exoplanet_modeling_and_analysis_center': 932, 'goddard_institute_for_space_studies': 933, 'earth_science_decadal_surveys': 934, 'exoplanet_opacities_database': 935, 'interactive_multiinstrument_database_of_solar_flares': 936, 'general_coordinates_network_gcn': 937, 'gcn_missions_instruments_and_facilities': 938, 'gcn_circulars': 939, 'igwn_public_alerts_user_guide': 940, 'algorithm_theoretical_basis_documents': 941, 'eos_mission_page': 942, 'earth_observer_publications': 943, 'our_changing_planet_the_view_from_space_images': 944, 'nasa_global_climate_change': 945, 'giss_publication_list': 946, 'giss_software_tools': 947, 'giss_datasets_and_derived_materials': 948, 'nasa_wavelength': 949, 'my_nasa_data': 950, 'mars_target_encyclopedia_mte': 951, 'astrogeology_analysis_ready_data': 952, 'nasa_science_missions_earth': 953, 'f_prime': 954, 'nasa_sea_level_change': 955, 'earth_observing_dashboard': 956, 'fire_information_for_resource_management_system_firms': 957, 'nasa_carbon_monitoring_system': 958, 'nasa_2023_climate_strategy': 959, 'navo_registry': 962, 'viewmaster_data_previews': 1028, 'coordinate_calculator': 963, 'extinction_calculator': 964, 'velocity_calculator': 965, 'high_level_science_products': 966, 'mast_web_services': 967, 'mast_api_search': 968, 'astroquery_api_search_mast_queries': 969, 'exoplanet_atmosphere_observability_table': 970, 'pan_starrs_catalog': 971, 'PDS_Cassini_Mission_Iapetus_(Saturn_VIII)_Website': 1204, 'pan_starrs_catalog_api': 972, 'mast_query_casjobs': 973, 'exo_mast': 974, 'mast_portal': 975, 'virtual_observatory_information': 976, 'co_plotter': 977, 'z_mast_search': 978, 'hubble_source_catalog_search': 979, 'space_telescope_bibliographic_search': 980, 'archived_synthetic_data': 981, 'viewmaster_data_diagrams': 1029, 'spectral_classes_of_like_stars': 982, 'skiff_spectral_catalog_search': 983, 'irsa_api': 984, 'herschel_data_search': 985, 'swas_spectrum_server': 986, 'swas_api_search': 987, 'atlas_api_search': 988, 'ssa_queries': 989, 'PDS_Cassini_Mission_Mimas_(Saturn_I)_Website': 1205, 'vo_simple_cone_search': 990, 'irsa_votable_access_protocol': 991, '2mass_interactive_image_service': 992, '2mass_image_inventory_search': 993, '2mass_batch_image_service': 994, '2mass_api_search': 995, 'image_and_spectrum_server_atlas': 996, 'ipac_table_validator': 997, 'galactic_dust_reddening_and_extinction': 998, 'hires': 1008, 'background_model': 999, 'irsa_idl_tools': 1000, 'image_validation': 1001, 'object_coordinate_lookup': 1002, 'montage_mosaic_engine': 1003, 'planck_data_tools': 1004, 'most': 1005, 'wise_neowise_coadder': 1006, 'scanpi': 1007, 'heasarc_browse_batch_interface': 1009, 'heasarc_download_scripts': 1010, 'vao_datascope': 1011, 'exoplanet_follow_up_observing_program': 1012, 'exofop_k2_campaign_9': 1013, 'koa_program_friendly_image_access_service': 1014, 'pykoa': 1015, 'neid_archive': 1016, 'viewmaster_data_calibrated': 1027, 'PDS_Cassini_Mission_Rhea_(Saturn_V)_Website': 1206, 'neid_solar_radial_velocity_archive': 1017, 'lbti_archive': 1018, 'corot_exoplanet_archive_etss_data_sets': 1019, 'contributed_datasets_in_the_exoplanet_archive': 1020, 'mast': 1021, 'nexsci': 1022, 'irsa': 1023, 'mast_hubble_search': 1024, 'planetary_image_galleries': 1025, 'viewmaster_data_volumes': 1026, 'viewmaster_data_archives_volumes': 1030, 'viewmaster_data_archives_calibrated': 1031, 'viewmaster_data_archives_previews': 1032, 'viewmaster_data_archives_diagrams': 1033, 'viewmaster_documents': 1034, 'venus_data_archive': 1035, 'small_bodies_node_data_by_type_of_observation': 1036, 'small_bodies_node_data_by_target_type': 1037, 'ftp_access': 1038, 'index': 1039, 'pds_geosciences_node_community': 1040, 'data_set_status': 1041, 'pds_data_dictionary_search': 1042, 'PDS_Cassini_Mission_Saturn_Small_Satellites_Website': 1207, 'PDS_Cassini_Mission_Tethys_(Saturn_III)_Website': 1208, 'ASTRO_Astrophysics_Documents_Website': 1043, 'ASTRO_Exoplanet_Program_Documents_Website': 1044, 'ASTRO_NED_User_Guides_Website': 1045, 'ASTRO_exoMAST_API_Website': 1046, 'ASTRO_MAST_Documentation_Website': 1047, 'ASTRO_ZMAST_API_Website': 1048, 'ASTRO_Hubble_Source_Catalog_Search_API_Website': 1049, 'ASTRO_High-Energy_Missions_Website': 1062, 'PDS_Cassini_Resource_Page_Website': 1209, 'ASTRO_Image_Cutouts_Website': 1050, 'ASTRO_Spitzer_Tools_Website': 1051, 'ASTRO_Data_Hosted_on_LAMBDA_Website': 1052, 'ASTRO_HEASARC_Software_Website': 1053, 'ASTRO_HEASARC_Tools_Website': 1054, 'ASTRO_Missions_and_Data_Website': 1063, 'PDS_CRISM_Analysis_Toolkit_(CAT)_Website': 1210, 'PDS_CRISM_Spectral_Library_Website': 1211, 'ASTRO_Calibration_Documentation_Website': 1055, 'ASTRO_Data_Reduction_Tools_Website': 1056, 'ASTRO_Contributed_Datasets_Website': 1057, 'ASTRO_HIRES_PRV_Website': 1058, 'ASTRO_NASA_Exoplanet_Archive_Documents_Website': 1059, 'ASTRO_TAP_Search_Website': 1060, 'ASTRO_API_Search_Website': 1061, 'PDS_Data_Archive_Website': 1212, 'PDS_Data_Portal_Website': 1213, 'ASTRO_James_Webb_Space_Telescope_Website': 1064, 'ASTRO_NAVO_HEASARC': 1065, 'PDS_Planetary_Data_System_(PDS)_Website': 1066, 'PDS_Astropedia_Lunar_and_Planetary_Cartographic_Catalog_Website': 1067, 'PDS_Astromat_Astromaterials_Data_System_Website': 1068, 'PDS_ISIS_Astro_Website': 1069, 'PDS_Java_Mission-planning_and_Analysis_for_Remote_Sensing_(JMARS)_Website': 1070, 'PDS_Wind_Tunnel_Particle_Threshold_Speed_Data_Website': 1096, 'PDS_Astromaterials_Acquisition_and_Curation_Office_Website': 1071, 'PDS_High-Resolution_Transmission_Molecular_Absorption_Database_(HITRAN)_Website': 1072, 'PDS_NASA_Space_Science_Data_Coordinated_Archive_(NSSDCA)_Website': 1073, 'PDS_NASA_Solar_System_Treks_Website': 1074, 'PDS_Mars_Exploration_Program_Website': 1075, 'PDS_NASA_Science_Solar_System_Exploration_Website': 1076, 'PDS_NASA_Science_Earths_Moon_Website': 1077, 'PDS_NASAs_Eyes_Website': 1078, 'CASEI_Platform': 1165, 'PDS_Solar_System_Exploration_Research_Virtual_Institute_(SSERVI)_Website': 1079, 'PDS_Ring-Moon_Systems_Node_On-line_Tools_Website': 1080, 'PDS_Toolkits_Website': 1081, 'PDS_Data_Volumes_Website': 1082, 'PDS_Errata_Website': 1083, 'PDS_Annex_Data_Holdings_Website': 1084, 'PDS_Missions_Archive_Page_Website': 1085, 'PDS_PPI_Software_Website': 1086, 'PDS_PPI_Documents_Website': 1087, 'LSDA_Website_Trial2': 1199, 'PDS_Archive_Navigator_Website': 1200, 'PDS_Image_Atlas_Website': 1088, 'PDS_PDS_Annex_Products_Website': 1089, 'PDS_Data_Pilot_Website': 1090, 'PDS_Imaging_Software_Website': 1091, 'PDS_PDS_Documentation_Website': 1092, 'PDS_All_Data_Holdings_Website': 1093, 'PDS_Data_Volumes_Index_Website': 1094, 'PDS_Atmospheric_Escape_Chemistry_Page_Website': 1095, 'PDS_Astrogeology_Website': 1201, 'PDS_Pluto_and_Arrokoth_Data_Archive_Website': 1097, 'PDS_Neptune_Archive_Page_Website': 1098, 'PDS_Uranus_Data_Archive_Website': 1099, 'PDS_Titan_Data_Archive_Website': 1100, 'PDS_Saturn_Data_Archive_Website': 1101, 'PDS_Outer_Planets_Icy_Satellites_Archive_Page_Website': 1102, 'PDS_Jupiter_Data_Archive_Website': 1103, 'PDS_Juno_Archive_Page_Website': 1104, 'PDS_Mars_Orbiter_Data_Website': 1105, 'PDS_Mars_Lander_Data_Website': 1106, 'PDS_Lunar_Atmospheres_Data_Archive_Website': 1107, 'PDS_Venus_Archive_Page_Website': 1108, 'PDS_Mercury_Data_Archive_Website': 1109, 'PDS_Messenger_MASCS_UVVS_Archive_Page_Website': 1110, 'PDS_Collision_Induced_Absorption_Model_Website': 1111, 'PDS_Recently_Archived_Volumes_Website': 1112, 'PDS_PDS_Atmospheres_Data_Set_Catalog_Website': 1113, 'PDS_Current_Missions_Website': 1114, 'PDS_Object_Access_Library_Website': 1115, 'PDS_Geosciences_Data_Holdings_Website': 1124, 'PDS_Cassini_Mission_Dione_(Saturn_IV)_Website': 1202, 'PDS_ISIS_Website': 1116, 'PDS_Mars_GCM_Website': 1117, 'PDS_EPIC_Model_Website': 1118, 'PDS_SBN_Tools_Utilities_and_Interfaces_Website': 1119, 'PDS_Small_Bodies_Image_Browser_Website': 1120, 'PDS_Mission_Data_Archive_Website': 1121, 'PDS_Small_Bodies_Data_Ferret_Website': 1122, 'PDS_Analyst_Notebook_Website': 1123, 'PDS_Gravity_Models_Website': 1125, 'PDS_Planetary_Science_Tools_Website': 1126, 'PDS_Virtual_Astronaut_Website': 1127, 'PDS_ODE_REST_Service_Website': 1128, 'PDS_Odyssey_GRS_Data_Node_Website': 1129, 'PDS_TES_Data_Node_Website': 1130, 'PDS_Geosciences_Node_Spectral_Library_Website': 1131, 'PDS_MRO_Coordinated_Observation_Website': 1132, 'PDS_Mars_Orbital_Data_Explorer_Website': 1133, 'PDS_LOLA_RDR_Query_Website': 1134, 'PDS_Lunar_Orbiter_Data_Explorer_Website': 1135, 'PDS_LOLA_RDR_Query_V20_Website': 1136, 'PDS_DIVINER_RDR_Query_Website': 1137, 'PDS_DIVINER_RDR_Query_V20_Website': 1138, 'PDS_Cassini_Mission_Enceladus_(Saturn_II)_Website': 1203, 'PDS_Mercury_Orbital_Data_Explorer_Website': 1139, 'PDS_Venus_Orbital_Data_Explorer_Website': 1140, 'PDS_Subscription_Service_Website': 1141, 'PDS_PDS_Tool_Registry_Website': 1142, 'PDS_PDS4_JParser_Website': 1143, 'PDS_PDS4_Local_Data_Dictionary_Tool_Website': 1144, 'PDS_PDS4_Training_Documents_Website': 1145, 'PDS_PDS4_Documents_Website': 1146, 'PDS_PDS3_Standards_Reference_Website': 1147, 'PDS_Metadata_Injector_for_PDS_Labels_Website': 1148, 'PDS_Photojournal_Website': 1149, 'PDS_Operational_Flight_Other_Project_Kernels_Website': 1150, 'PDS_Generic_Kernels_Website': 1151, 'PDS_SPICE_Archives_Website': 1152, 'PDS_SPICE_Utility_and_Application_Programs_Website': 1153, 'PDS_SPICE_Toolkit_Website': 1154, 'CASEI_Instrument': 1164, 'PDS_SPICE_Toolkit_Documentation_Website': 1155, 'PDS_SPICE-enhanced_Cosmographia_Website': 1156, 'PDS_SPICE_Tutorials_Website': 1157, 'PDS_SPICE_Programming_Lessons_Website': 1158, 'PDS_SPICE_Self-Training_Website': 1159, 'PDS_API_Legacy_All': 1160, 'ARSETAppliedSciences': 1161, 'CASEI_Campaign': 1162, 'CASEI_Deployment': 1163, 'CEOS_API_I': 1166, 'CEOS_API_M': 1167, 'CMR_API': 1168, 'DataPathFinder': 1169, 'NasaEarthObservationWebsite': 1170, 'NTRS': 1171, 'Autoplot_Website': 1172, 'CCMC_Website': 1173, 'HAPI_Website': 1174, 'Helio_Events_Knowledgebase_Website': 1175, 'Helioviewer_Website': 1176, 'PyHC_Website': 1177, 'SPASE_Website': 1178, 'SPEDAS_Website': 1179, 'GENELAB_Github_DataProcessing': 1180, 'GENELAB_Github_SampleProcessing': 1181, 'GENELAB_Github_Training': 1182, 'GENELAB_METADATA_Website': 1183, 'GENELAB_Publications_Website': 1184, 'LSDA_Website': 1185, 'ASTRO_Finder_Chart_Website': 1186, 'ASTRO_Multi_Website': 1187, 'ASTRO_Planck_Cutout_Visualization_Website': 1188, 'ASTRO_WISE_Image_Service_Website': 1189, 'CASEI': 1190, 'CODE_NASA_API': 1191, 'ESSCOR_API': 1192, 'GCIS_ARTICLE_API': 1193, 'GCIS_BOOKS_API': 1194, 'GCIS_JOURNAL_API': 1195, 'GCIS_REPORTS_API': 1196, 'Helioviewer_Documentation_Website': 1197, 'LSDA_Website_Trial': 1198, 'PDS_Dawn_Mission_to_Ceres_Website': 1214, 'PDS_Dawn_Mission_to_Vesta_Website': 1215, 'PDS_LADEE_NMS_Calibrated_Data_Search': 1216, 'PDS_LADEE_NMS_Derived_Data_Search': 1217, 'PDS_LADEE_UVS_Calibrated_Data_Search': 1218, 'PDS_Map-a-Planet_(MAP)_Website': 1219, 'PDS_MAVEN_ACC_Data_Search': 1220, 'PDS_MAVEN_NGIMS_Data_Search': 1221, 'PDS_Missions_Website': 1222, 'PDS_Models_and_Simulations_Website': 1223, 'PDS_Near_Earth_Asteroid_Rendezvous_(NEAR)_Data_Archive_Website': 1224, 'PDS_NEAR_Shoemaker_Mission_to_433_Eros_Website': 1225, 'PDS_New_Horizons_Encounter_with_Pluto_Website': 1226, 'PDS_Niels_Bohr_Institute_Website': 1227, 'PDS_Notebook_Website': 1228, 'PDS_OMEGA_Analysis_Toolkit_(OAT)_Website': 1229, 'PDS_OPUS_Website': 1230, 'PDS_OSIRIS-REx_Mission_to_Bennu_Website': 1231, 'PDS_PDS_Small_Bodies_Node_Asteroid_Dust_Subnode_Website': 1232, 'PDS_PDS_Software_Tools_Tutorial_and_Viewers_Website': 1233, 'PDS_Phoebe_Saturn_IX_Website': 1234, 'PDS_Projection_on_the_Web_(POW)_Service_Website': 1235, 'PDS_Rings_Website': 1236, 'PDS_SBIB_3D_Website': 1237, 'PDS_Subset_Tool_Website': 1238, 'PDS_Users_Guides_Website': 1239, 'PDS_USGS_Pilot_Website': 1240, 'PDS_Web_Chronos_Website': 1241, 'TASKBOOK_Website': 1242}

In [38]:
# handle other folder: sinequa_scraped
import json

kaylin_sources = glob.glob('source_configurations/sinequa_scraped/*.*')
def generate_json(row, config_folder):
    return {
        "model": "sde_collections.candidateurl",
        "fields": {
            "collection": collection_ids[config_folder],
            "url": row.url,
            "scraped_title": row.title,
            "generated_title": "",
            "level": 2,
            "visited": False
        }
    }
replacement_dict = {
    'pds4_documents': 'PDS_PDS4_Documents_Website',
    'heliophysics_events_knowledgebase': 'Helio_Events_Knowledgebase_Website',
    'mars_target_encyclopedia': 'mars_target_encyclopedia_mte',
    'pds_near_earth_asteroid_rendezvous_near_data_archive_website': 'PDS_Near_Earth_Asteroid_Rendezvous_(NEAR)_Data_Archive_Website',
    'solar_system_exploration_research_virtual_institute_sservi': 'PDS_Solar_System_Exploration_Research_Virtual_Institute_(SSERVI)_Website',
    'astromaterials_acquisition_and_curation_office': 'PDS_Astromaterials_Acquisition_and_Curation_Office_Website',
    'mars_exploration_program': 'PDS_Mars_Exploration_Program_Website',
    'nasa_science_earths_moon': 'PDS_NASA_Science_Earths_Moon_Website',
}
for source in kaylin_sources:
    file_name = os.path.basename(source)
    config_folder, _ = os.path.splitext(file_name)

    if "SPASE_JSON_List" in file_name:
        continue

    with open(source, "r") as f:
        directory = f'../scraper/scraped_urls/{config_folder}'
        if not os.path.exists(directory):
            os.makedirs(directory)
        json_file_name = f"{directory}/urls.json"

        if not config_folder in collection_ids:
            try:
                config_folder = replacement_dict[config_folder]
            except KeyError:
                print(f"Skipping {config_folder}")
                continue

        df = pd.read_csv(
            source,
            sep="\~\?\~",
            header=None,
            skiprows=[0],
            names=["url", "Rel folder", "title"],
        )
        df = df[["url", "title"]]
        try:
            df = df.drop_duplicates(subset=['url'])
            if config_folder == 'PDS_NASA_Science_Earths_Moon_Website':
                df = df[df.url.str.startswith('http')]
            df['json'] = df.apply(generate_json, args=(config_folder, ), axis=1)

            with open(json_file_name, "w") as f:
                f.write(json.dumps(df['json'].to_list(), indent=4))
        except KeyError:
            print(f"Some key error on: {config_folder}")

/tmp/ipykernel_32301/1953748084.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
/tmp/ipykernel_32301/1953748084.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
/tmp/ipykernel_32301/1953748084.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(
/tmp/ipykernel_32301/1953748084.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' 

In [39]:
def generate_ingestion_commands():
    folders = glob.glob('../scraper/scraped_urls/*/')
    for folder in folders:
        folder = os.path.basename(os.path.normpath(folder))
        print(f"python manage.py loaddata scraper/scraped_urls/{folder}/urls.json")

generate_ingestion_commands()

python manage.py loaddata scraper/scraped_urls/heliophysics_events_knowledgebase/urls.json
python manage.py loaddata scraper/scraped_urls/my_nasa_data/urls.json
python manage.py loaddata scraper/scraped_urls/exoplanet_opacities_database/urls.json
python manage.py loaddata scraper/scraped_urls/earth_observing_dashboard/urls.json
python manage.py loaddata scraper/scraped_urls/PDS_Astropedia_Lunar_and_Planetary_Cartographic_Catalog_Website/urls.json
python manage.py loaddata scraper/scraped_urls/nasa_sea_level_change/urls.json
python manage.py loaddata scraper/scraped_urls/goddard_institute_for_space_studies/urls.json
python manage.py loaddata scraper/scraped_urls/nasa_power/urls.json
python manage.py loaddata scraper/scraped_urls/general_coordinates_network_gcn/urls.json
python manage.py loaddata scraper/scraped_urls/gcn_missions_instruments_and_facilities/urls.json
python manage.py loaddata scraper/scraped_urls/pds_near_earth_asteroid_rendezvous_near_data_archive_website/urls.json
pytho